# Votação paralela

In [8]:
from multiprocessing import Pool
import time
import math

Import Starknet libs.

In [2]:
from starknet_py.net.account.account import Account
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.signer.stark_curve_signer import KeyPair
from starknet_py.net.models.chains import StarknetChainId
from starknet_py.contract import Contract


Import Parllier library and generate keys.

In [ ]:
from phe import paillier

public_key, private_key = paillier.generate_paillier_keypair(n_length=20)



[253288798591, 203305245440]
n:  563903


In [68]:
# Initial state
secret_number_list: list[int] = [0, 0]
encrypted_number_list = [public_key.encrypt(x) for x in secret_number_list]

print(list(map(lambda enc: enc.ciphertext(), encrypted_number_list)))
print("n: ", public_key.n)

[261827510938, 207975397875]
n:  563903


Read accounts.

In [ ]:
import json

with open("../stjs/accounts.json") as accounts_file:
    accounts = json.load(accounts_file)
    
accounts

{'accounts': [{'publicKey': '0x63b1357f07da619f5a04024561c2249eee2d678a2b118420b3c1f6c459bb29b',
   'privateKey': '0x6e6a14b187d5d4a4f3bf6029f90c5e138b7decfa7e32009c32823b923c037bd',
   'address': '0x28ca818f2e7f75b717a35cfbbe4d02c859f22833bbd585e5fb3ab560ed93690'},
  {'publicKey': '0x27fbb4baac0c173eada558cac51a190ca2765f7616bd31c9e46349e197fd479',
   'privateKey': '0x3c1ac96a203e5de78cab3fe86209d3e20b80103efb21e91fb3b174198d5dd28',
   'address': '0x5cd2bfc4e37d1f9b8d43967f616cc4e440744e732f0de725c42533ba8eea22c'},
  {'publicKey': '0x5976942599a6ecd50868d796eb69ae20fde0666fe8b6e6fb858994f5c3bdb97',
   'privateKey': '0x3920c75d7439e5aae812ffe966862324bfcac422d6a7b08031bb81e174294f8',
   'address': '0x60fbbc26f81c44956b2d284f5de23ffc00309fe4d8e6abbf9821d71d601dcfe'},
  {'publicKey': '0x21829d587e9794cb0dd1598915cab068ed1d8a2d18dc37000f19f64ebeb2a46',
   'privateKey': '0x576b9757eddcb15b67d58c719be17158337e061925563eb4aec27486173ac8a',
   'address': '0x54f311d99bb589f41795db1d1651d1f0c96

Create client.

In [ ]:
client = FullNodeClient(node_url='https://starknet-sepolia.infura.io/v3/4deda563b65c4a7c92f5c57b43cd4b29')

Transaction hashes array.

In [38]:
txHashes: list[str] = []

Function for voting.

In [54]:
async def voting(
    contract_address: str, 
    account_obj: dict, 
    client: FullNodeClient, 
    no_vote: int, 
    yes_vote: int
):    
    account = Account(
        client=client,
        address=account_obj["address"],
        key_pair=KeyPair(
            private_key=account_obj["privateKey"], 
            public_key=account_obj["publicKey"]
        ),
        chain=StarknetChainId.SEPOLIA,
    )
    
    print("Voting account with address " + account_obj["address"] + "\n")
    
    contract = await Contract.from_address(address=(contract_address), provider=account)
    
    start_time = time.perf_counter()
    invocation = await contract.functions["vote"].invoke_v1([no_vote, yes_vote], max_fee=int(1e14))
    end_time = time.perf_counter()
    
    print("Invocation: " + str(hex(invocation.hash)) + ", " + str(invocation.status) + "\n")
    
    return {
        "tx_hash": hex(invocation.hash),
        "tx_time": end_time - start_time
    }

Parallelize.

In [ ]:
import random
from asyncio import run

def lambda_runner(account): 
    yes_vote: int = random.randint(0,1)
    no_vote: int = 0 if yes_vote == 1 else 1
    
    yes_ciphered_vote: int = public_key.encrypt(yes_vote).ciphertext()
    no_ciphered_vote: int = public_key.encrypt(no_vote).ciphertext()
    
    return {
        "execution": run(
            voting(
                    contract_address="0x13cd4bd645d6908d25cd353937c7c43bcde36b18691e6754eb487383eb1ecef", 
                    account_obj= account, 
                    client=client,
                    no_vote=no_ciphered_vote,
                    yes_vote=yes_ciphered_vote
                )
        ),
        "votes_table": {
            "yes": yes_vote,
            "no": no_vote,
            "yes_ciphered": yes_ciphered_vote,
            "no_ciphered_vote": no_ciphered_vote
        }
    }

with Pool(100) as pool:
        result = pool.map(lambda_runner, accounts["accounts"][0:100])
        
result

Voting account with address 0x54f311d99bb589f41795db1d1651d1f0c96141394cd37a14a349ecc325e80e5
Voting account with address 0x171bccff4b09d7d84b93ed4f9fbaff26e2a527d5c6b8f3c2801d7113e53b90b
Voting account with address 0x60fbbc26f81c44956b2d284f5de23ffc00309fe4d8e6abbf9821d71d601dcfe
Voting account with address 0x5c860e7449ce443dc6994b7c383e751b620d9f1983f18be14be94194d7dcb54
Voting account with address 0x1d770108eef948e8c99807137b68a768090f50ddad45a11f5dbc21216220917
Voting account with address 0x4da4080dccd78dbc7b8cfd115df1145a841272806d703814da198e8239003ee
Voting account with address 0x30e5de6724a469c244c5294e88ef7645235518a17a92d219d10105ed9ae6e2a
Voting account with address 0x206a0d8499f23da2773acba70d55028c45a4075d248d8320a10ad1437d54ed
Voting account with address 0xe00384ea8fd8b22f373bc5f13f725c8659375e30a28937ddb725b01ab75bfc
Voting account with address 0x5cd2bfc4e37d1f9b8d43967f616cc4e440744e732f0de725c42533ba8eea22c
Voting account with address 0x28ca818f2e7f75b717a35cfbbe4d02c8

[{'execution': {'tx_hash': '0xca51395aa429062dece72dab2667a7538609985c3b1ac9344b8d30333a28c',
   'tx_time': 17.028890654997667},
  'votes_table': {'yes': 0,
   'no': 0,
   'yes_ciphered': 255507239963,
   'no_ciphered_vote': 196783409635}},
 {'execution': {'tx_hash': '0x1262c72ceb83fa4674130f1a9ae75820dbe631c10bda5136088cb07b1aab2f3',
   'tx_time': 16.003934802000003},
  'votes_table': {'yes': 0,
   'no': 1,
   'yes_ciphered': 46424175149,
   'no_ciphered_vote': 71341939383}},
 {'execution': {'tx_hash': '0x74d8f684751338d791bba12821851e29f122dd43f6be62991c206729f62771f',
   'tx_time': 13.165585394999653},
  'votes_table': {'yes': 0,
   'no': 1,
   'yes_ciphered': 146619707873,
   'no_ciphered_vote': 301558033675}},
 {'execution': {'tx_hash': '0x23684e2306b0e34fc6fdf2d7f20e0937b9d19c3e1f8dc4b8089ca5fe1d2c8f3',
   'tx_time': 14.91439508300391},
  'votes_table': {'yes': 0,
   'no': 1,
   'yes_ciphered': 299816857631,
   'no_ciphered_vote': 200257243270}},
 {'execution': {'tx_hash': '0xbdc

Write tx hashes in file.

In [71]:
with open("results.json", "w") as out:
    out.write(
        json.dumps(
            { "results": result }
        )
    )

In [33]:
encrypted_result = [337214314798, 28868045133]
decrypted_result = [private_key.decrypt(paillier.EncryptedNumber(public_key, ciphered_result)) for ciphered_result in encrypted_result]
decrypted_result

[3, 2]